# Incontro ISLAB 2017-03-23 | Esercitazione

### Esercitazione sul nostro dataset

- costruzione di un'indice delle risorse (tecnologia a piacere)
- indice deve essere velocemente accessibile rispetto al dizionario (**tag**)
- indice deve essere idealmente velocemente accessibile rispetto al documento


In [52]:
ata_settings = {
    'instagram': {
        'date_pattern': '%d/%m/%Y %H:%M',
        'tags': 'string',
        'integer_fields': ['comments_count', 'likes_count', 'total']
    },
    'facebook': {
        'date_pattern': '%Y-%m-%dT%H:%M:%S',
        'tags': 'list',
        'integer_fields': ['comments_count', 'likes_count', 'total', 'shares_count']
        
    }
}

In [1]:
import pymongo
import numpy as np


db = pymongo.MongoClient()['mec']
dataset = 'calzedoniajson_items_0'

df, tf = {}, {}
docs = {}

def process_tf(i, j, tag, tags, probs):
    try:
        posting = tf[tag]
    except KeyError:
        posting = {}
    posting[i] = {
        'raw_tf': 1, 'prob': probs[j],
        'norm_p': probs[j]/max(probs),
        'freq': 1.0 / len(tags)
    }
    tf[tag] = posting
    return

def process_df(tag):
    try:
        df[tag]['df'] += 1
    except KeyError:
        df[tag] = {'df': 1.0}

def calculate_idf(nt, size):
    return np.log(1 + ((float(size)-nt) / nt))

def process_idf(tag, size):
    df[tag]['idf'] = calculate_idf(df[tag]['df'],size)

def process_tag(i, j, tag, tags, probs, size):
    process_tf(i, j, tag, tags, probs)
    process_df(tag)
    process_idf(tag, size)

cursor = db[dataset].find(no_cursor_timeout=True).sort('_id', pymongo.ASCENDING) # optional no_cursor_timeout=True
size = cursor.count()
for i, doc in enumerate(cursor) :
    docs[i] = {'id': doc['_id'], 'tags':[]}
    try:
        tags = doc['clarifai_tags']
        docs[i]['tags'] = tags
        probs = doc['clarifai_probs']
        [process_tag(i, j, tag, tags, probs, size) for j, tag in enumerate(tags)]
    except KeyError:
        pass
cursor.close()


<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 'dict'>
<type 

In [54]:
def empty_and_insert(collection_name, data):
    if db[collection_name].count() > 0:
        db[collection_name].drop()
    db[collection_name].insert_many(data)
    return db[collection_name].count()

docs_map = [dict({'posting_key': k}.items() + v.items()) for k,v in docs.items()]
print empty_and_insert('calzedoniajson_docs_0', docs_map)

df_map = [dict({'tag': k}.items() + v.items()) for k,v in df.items()]
print empty_and_insert('calzedoniajson_df_0', df_map)

tf_map = [dict({'tag': k}.items() + v.items()) for k,v in df.items()]
print empty_and_insert('calzedoniajson_tf_0', df_map)



450
694
694
